<a href="https://colab.research.google.com/github/AntoPrinzi99/NN-project-Antonino-Prinzivalli/blob/main/TEXT_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install pytorch_lightning
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [9]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModel, AutoTokenizer, MarianMTModel, MarianTokenizer
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from torch.optim import Adam
from datasets import load_dataset
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel, RobertaTokenizer, AutoModel, AutoTokenizer


In [20]:
# Determine the device to use (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models and tokenizers

In [4]:
# Loading models and tokenizers
english_model = RobertaModel.from_pretrained("roberta-base").to(device)
english_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
italian_model = AutoModel.from_pretrained("dbmdz/bert-base-italian-uncased").to(device)
italian_tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/243k [00:00<?, ?B/s]

i did use this below to create the translations of the sentence of the italian anchors , since the anchors should have the same meaning

In [ ]:
#from transformers import MarianMTModel, MarianTokenizer
#from datasets import load_dataset
#import torch

## Load translation model and tokenizer for English to Italian
#def load_translation_model():
#    model_name = 'Helsinki-NLP/opus-mt-en-it'
#    model = MarianMTModel.from_pretrained(model_name)
#    tokenizer = MarianTokenizer.from_pretrained(model_name)
#    return model, tokenizer

## Translate a list of sentences using the Marian model
#def translate_sentences(sentences, model, tokenizer):
#    translated_sentences = []
#    # Tokenize sentences (with padding and truncation)
#    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
#    # Generate translation outputs
#    with torch.no_grad():
#        outputs = model.generate(**inputs)
#    # Decode translated sentences
#    for output in outputs:
#        translated_sentence = tokenizer.decode(output, skip_special_tokens=True)
#        translated_sentences.append(translated_sentence)
#    return translated_sentences

# Load datasets
#train_dataset_en = load_dataset("tyqiangz/multilingual-sentiments", 'english')

# Load sample English sentences
#english_sentences = train_dataset_en['train']['text'][:768]  # Adjust sample size as needed

# Load models
#translation_model, translation_tokenizer = load_translation_model()

# Translate to Italian
#italian_translations = translate_sentences(english_sentences, translation_model, translation_tokenizer)

## Output the first few translations for review
#for en, it in zip(english_sentences[:10], italian_translations[:10]):
#    print(f"EN: {en} -> IT: {it}")

In [6]:
def load_translations(file_path):
    english_sentences = []
    italian_translations = []

    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for i in range(0, len(lines), 3):  # Each entry has 3 lines: EN, IT, and a blank line
            if i + 1 < len(lines):  # Ensure there's an Italian line following the English line
                en_line = lines[i].strip()
                it_line = lines[i + 1].strip()
                if en_line.startswith("EN: ") and it_line.startswith("IT: "):
                    english_sentence = en_line[4:]  # Remove the "EN: " prefix
                    italian_translation = it_line[4:]  # Remove the "IT: " prefix
                    english_sentences.append(english_sentence)
                    italian_translations.append(italian_translation)

    return english_sentences, italian_translations

# Load translations from file
english_sentences, italian_translations = load_translations('translations.txt')

# Output the first few translations for review
for en, it in zip(english_sentences[:10], italian_translations[:10]):
    print(f"EN: {en} -> IT: {it}")

EN: okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\u2019mon America aren\u2019t you sick of her yet? (sorry) -> IT: Okay i\u2019m dispiaciuto ma TAYLOR SWIFT non sembra come Jackie o così smettere di confrontare i due. c\u2019mon America aren\u2019t ancora stufo di lei? (scusa)
EN: @user the DC comics site has Batman 44 releases on the 9th but its out now? -> IT: @user il sito di fumetti DC ha Batman 44 uscite il 9 ma la sua uscita ora?
EN: "Frank Gaffrey\u002c Cliff May\u002c Steve Emerson: Brilliant. \""""Looming Threats: Iran\u002c Hezbollah Hamas\"""" is the best #cufidc session I\u2019ve had thus far." -> IT: "Frank Gaffrey\u002c Cliff May\u002c Steve Emerson: Brilliant. \""""Looming Threats: Iran\u002c Hezbollah Hamas\""""" è la miglior sessione #cufidc che abbia avuto finora."
EN: The tragedy of only thinking up hilarious tweets for the Summer Olympics now is that in four years there may be no place for them. -> IT: La tragedia di p

data:

In [52]:
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Caricamento dei dati
dataset = load_dataset("tyqiangz/multilingual-sentiments", 'english')
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
train_data = TextDataset(train_texts, train_labels)
data_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [60]:
sample_text, sample_label = train_data[0]
print("Sample Text:", sample_text)
print("Sample Label:", sample_label)

Sample Text: okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\u2019mon America aren\u2019t you sick of her yet? (sorry) 
Sample Label: 2


In [91]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

    def relative_projection(self, x, anchors):
        x_norm = F.normalize(x, p=2, dim=-1)
        anchors_norm = F.normalize(anchors, p=2, dim=-1)
        return torch.einsum("bm, am -> ba", x_norm, anchors_norm)

def relative_projection(x, anchors):
    x_norm = F.normalize(x, p=2, dim=-1)
    anchors_norm = F.normalize(anchors, p=2, dim=-1)
    return torch.einsum("bm, am -> ba", x_norm, anchors_norm)


def compute_embeddings(model, tokenizer, texts, device):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

no trasformation here:

In [110]:
def train_classifier(model, tokenizer, data_loader, anchors_embeddings, device):
    # Defining the classifier and moving to the appropriate device
    classifier = TextClassifier(input_dim=768, num_classes=3).to(device)
    optimizer = Adam(classifier.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    model.eval()  # Set the embedding model to evaluation mode
    classifier.train()  # Set the classifier to training mode

    for epoch in range(30):
        total_loss = 0
        for texts, labels in data_loader:
            # generate the embeddings
            embeddings = compute_embeddings(model, tokenizer, texts, device)

            # apply the relative trasformation
            #relative_embeddings = relative_projection(embeddings, anchors_embeddings)
            #print(relative_embeddings.shape)

            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = classifier(embeddings) # in this code we use the absolute embedding
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(data_loader)}')

    return classifier

In [24]:

def compute_embeddings(model, tokenizer, texts, device):
    model.eval()
    model.to(device)


    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():  # Disable gradient calculation to save resources during inference
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Calculates the average of the hidden states to obtain the embedding
    return embeddings

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#we will use this after
anchor_embeddings_en = compute_embeddings(english_model, english_tokenizer, english_sentences, device)
anchor_embeddings_it = compute_embeddings(italian_model, italian_tokenizer, italian_translations, device)


In [88]:
anchor_embeddings_it.shape

torch.Size([768, 768])

In [111]:
trained_classifier = train_classifier(english_model, english_tokenizer, data_loader, anchor_embeddings_en, device)#i pass anchor_embeddings_en but i don't use it in this absolute training

Epoch 1, Loss: 1.0524810419000428
Epoch 2, Loss: 0.901841611697756
Epoch 3, Loss: 0.8192891766285074
Epoch 4, Loss: 0.7767322227872652
Epoch 5, Loss: 0.7756410169190374
Epoch 6, Loss: 0.7261231726613538
Epoch 7, Loss: 0.7011837373519766
Epoch 8, Loss: 0.6659966404068058
Epoch 9, Loss: 0.6527412405301785
Epoch 10, Loss: 0.6248294135619854
Epoch 11, Loss: 0.6002736733905201
Epoch 12, Loss: 0.5820421108911777
Epoch 13, Loss: 0.5804214688210652
Epoch 14, Loss: 0.5488545979405272
Epoch 15, Loss: 0.5591263359990614
Epoch 16, Loss: 0.5187517635781189
Epoch 17, Loss: 0.5319446268780478
Epoch 18, Loss: 0.47169910191461956
Epoch 19, Loss: 0.4724209912378213
Epoch 20, Loss: 0.4744054469568976
Epoch 21, Loss: 0.411631738831257
Epoch 22, Loss: 0.3925033841153671
Epoch 23, Loss: 0.3697088311972289
Epoch 24, Loss: 0.36910324055573035
Epoch 25, Loss: 0.3345203661713107
Epoch 26, Loss: 0.35079404429115096
Epoch 27, Loss: 0.2782833172843374
Epoch 28, Loss: 0.27618916499717483
Epoch 29, Loss: 0.286536552

In [117]:
eval_dataset_en = load_dataset("tyqiangz/multilingual-sentiments", 'english')['test']
eval_dataset_it = load_dataset("tyqiangz/multilingual-sentiments", 'italian')['test']

In [118]:
test_texts = [example['text'] for example in eval_dataset_en]
test_labels = [example['label'] for example in eval_dataset_en]


test_dataset = TextDataset(test_texts, test_labels)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=True)

In [119]:
sample_text, sample_label = test_dataset[0]
print("Sample Text:", sample_text)
print("Sample Label:", sample_label)

Sample Text: Trying to have a conversation with my dad about vegetarianism is the most pointless infuriating thing ever #caveman 
Sample Label: 2


Stampa dei primi 5 elementi del dataset

In [120]:
for i in range(5):
    sample_text, sample_label = test_dataset[i]
    print(f"Sample {i+1} - Text: {sample_text} / Label: {sample_label}")

Sample 1 - Text: Trying to have a conversation with my dad about vegetarianism is the most pointless infuriating thing ever #caveman  / Label: 2
Sample 2 - Text: #latestnews 4 #newmexico #politics + #nativeamerican + #Israel + #Palestine  -  Protesting Rise Of Alt-Right At...  / Label: 1
Sample 3 - Text: @user You are a stand up guy and a Gentleman Vice President Pence  / Label: 0
Sample 4 - Text: @user @user @user Looks like Flynn isn't too pleased with me, he blocked me. You blocked by Flynn too @user  / Label: 2
Sample 5 - Text: perfect pussy clips #vanessa hudgens zac efron naked  / Label: 1


In [121]:
def evaluate_classifier(classifier, model, tokenizer, data_loader, anchors_embeddings, device):
    classifier.eval()
    model.eval()
    total, correct = 0, 0

    with torch.no_grad():
        for texts, labels in data_loader:
            # Generate embeddings for the texts
            embeddings = compute_embeddings(model, tokenizer, texts, device)
            # Apply the relative projection transformation
            #relative_embeddings = relative_projection(embeddings, anchors_embeddings)
            # Convert labels to the appropriate device
            labels = torch.tensor(labels).to(device)
            # Pass the transformed embeddings through the classifier
            outputs = classifier(embeddings)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')
    return accuracy

In [122]:
# Assume anchor_embeddings_en is already calculated and transferred to the correct device
accuracy_en = evaluate_classifier(trained_classifier, english_model, english_tokenizer, test_loader, anchor_embeddings_en, device)
print("Accuracy on English test dataset:", accuracy_en)

<ipython-input-121-fe8d881ac821>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Accuracy: 64.25%
Accuracy on English test dataset: 0.6425287356321839


now i try with the transformation :

In [101]:
def train_classifier(model, tokenizer, data_loader, anchors_embeddings, device):
    # Definizione del classificatore e spostamento sul dispositivo appropriato
    classifier = TextClassifier(input_dim=768, num_classes=3).to(device)
    optimizer = Adam(classifier.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    model.eval()  # Imposta il modello di embedding in modalità di valutazione
    classifier.train()  # Imposta il classificatore in modalità di addestramento

    for epoch in range(10):
        total_loss = 0
        for texts, labels in data_loader:

            embeddings = compute_embeddings(model, tokenizer, texts, device)
            # apply the trasformation
            relative_embeddings = relative_projection(embeddings, anchors_embeddings)

            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = classifier(relative_embeddings)  #NOW I PASS THE RELATIVE EMBEDDINGS
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(data_loader)}')

    return classifier

In [102]:
trained_classifier = train_classifier(english_model, english_tokenizer, data_loader, anchor_embeddings_en, device)

Epoch 1, Loss: 1.1063099914583667
Epoch 2, Loss: 1.101671054445464
Epoch 3, Loss: 1.09968625677043
Epoch 4, Loss: 1.1017154619611542
Epoch 5, Loss: 1.0998188503857316
Epoch 6, Loss: 1.0987594045441726
Epoch 7, Loss: 1.0995183340434371
Epoch 8, Loss: 1.0992310458216175
Epoch 9, Loss: 1.0983858560693676
Epoch 10, Loss: 1.0995952754185117


In [103]:
eval_dataset_en = load_dataset("tyqiangz/multilingual-sentiments", 'english')['test']
eval_dataset_it = load_dataset("tyqiangz/multilingual-sentiments", 'italian')['test']

In [ ]:

test_texts = [example['text'] for example in eval_dataset_en]
test_labels = [example['label'] for example in eval_dataset_en]


test_dataset = TextDataset(test_texts, test_labels)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=True)

In [104]:
sample_text, sample_label = test_dataset[0]
print("Sample Text:", sample_text)
print("Sample Label:", sample_label)

Sample Text: Trying to have a conversation with my dad about vegetarianism is the most pointless infuriating thing ever #caveman 
Sample Label: 2


Printing the first 5 elements of the dataset

In [105]:
for i in range(5):
    sample_text, sample_label = test_dataset[i]
    print(f"Sample {i+1} - Text: {sample_text} / Label: {sample_label}")

Sample 1 - Text: Trying to have a conversation with my dad about vegetarianism is the most pointless infuriating thing ever #caveman  / Label: 2
Sample 2 - Text: #latestnews 4 #newmexico #politics + #nativeamerican + #Israel + #Palestine  -  Protesting Rise Of Alt-Right At...  / Label: 1
Sample 3 - Text: @user You are a stand up guy and a Gentleman Vice President Pence  / Label: 0
Sample 4 - Text: @user @user @user Looks like Flynn isn't too pleased with me, he blocked me. You blocked by Flynn too @user  / Label: 2
Sample 5 - Text: perfect pussy clips #vanessa hudgens zac efron naked  / Label: 1


In [112]:
def evaluate_classifier(classifier, model, tokenizer, data_loader, anchors_embeddings, device):
    classifier.eval()
    model.eval()
    total, correct = 0, 0

    with torch.no_grad():
        for texts, labels in data_loader:

            embeddings = compute_embeddings(model, tokenizer, texts, device)
            # Apply the relative projection transformation
            relative_embeddings = relative_projection(embeddings, anchors_embeddings)
            # Convert labels to the appropriate device
            labels = torch.tensor(labels).to(device)
            # Pass the transformed embeddings through the classifier
            outputs = classifier(relative_embeddings) # NOW EVALUATE WITH THE TRANSFORMATION
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')
    return accuracy

In [107]:
accuracy_en = evaluate_classifier(trained_classifier, english_model, english_tokenizer, test_loader, anchor_embeddings_en, device)
print("Accuracy on English test dataset:", accuracy_en)

<ipython-input-106-8f9a54d0b5be>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Accuracy: 33.33%
Accuracy on English test dataset: 0.3333333333333333


now i use the classifier with the italian evaluation data set:

In [113]:
test_texts = [example['text'] for example in eval_dataset_it]
test_labels = [example['label'] for example in eval_dataset_it]

In [114]:
test_dataset = TextDataset(test_texts, test_labels)

In [115]:
sample_text, sample_label = test_dataset[0]
print("Sample Text:", sample_text)
print("Sample Label:", sample_label)

Sample Text: @user @user Ma Ferrero? il compagno Ferrero? ma il suo partito esiste ancora? allora stiamo proprio frecati !!!
Sample Label: 2


In [116]:
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accuracy_it = evaluate_classifier(trained_classifier, italian_model, italian_tokenizer, test_loader, anchor_embeddings_it, device)
print("Accuracy on Italian test dataset:", accuracy_en)

<ipython-input-112-8f9a54d0b5be>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Accuracy: 33.33%
Accuracy on Italian test dataset: 0.3333333333333333
